# Парсинг с помощью BeautifulSoup

Beautiful Soup - это библиотека для Python, которая позволяет парсить (анализировать) HTML и XML документы. Она предоставляет удобный способ искать, навигировать, и модифицировать дерево DOM (Document Object Model), представляющее HTML/XML документ.

# Задание

Вам необходимо собрать датасет, спарсив данные из этого сайта:

https://books.toscrape.com/

Всего на сайте 1000 книг. То есть длина датасета должна равняться количеству книг.
 
Итоговая таблица должна содержать следующие столбцы:

| Название столбца | Описание | 
|--|--|
|id| Идентификатор книги |
|book_name| Название книги |
|price| Цена в £ |
|stock| Наличие книги. 1 или 0|
|url| Ссылка на книгу |

**Примечание по столбцам:**
- `id` - заполняется разработчиком датасета. Первая спарсенная книга имеет `id` = `0`.
- `url` - должна содержать полную ссылку. Не только конец ссылки, указанный на сайте. То есть по данному url можно перейти одним кликом.

## Импорт библиотек

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

## Cоздание датасета и парсинг данных

In [2]:
# Функция для получения данных о книгах
def scrape_books():
    # Создаем пустой список для хранения данных о книгах
    books_data = []
    
    # Перебираем все страницы с книгами (50 страниц)
    for page_num in range(1, 51):
        url = f'http://books.toscrape.com/catalogue/page-{page_num}.html'
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Находим все элементы с информацией о книгах на странице
        books = soup.find_all('article', class_='product_pod')
        
        # Извлекаем данные о каждой книге
        for book in books:
            # Извлекаем id книги
            book_id = book.select_one('h3 a')['href'].split('_')[-1].split('.')[0]
            
            # Извлекаем название книги
            book_name = book.select_one('h3 a')['title']
            
            # Извлекаем цену книги и удаляем лишние символы
            price_str = book.select_one('p.price_color').get_text().replace('£', '').replace('Â', '')
            price = float(price_str)
            
            # Извлекаем наличие книги
            if 'In stock' in book.select_one('p.availability').get_text():
                stock = 1
            else:
                stock = 0
            
            # Создаем ссылку на книгу
            book_url = 'http://books.toscrape.com/catalogue/' + book.select_one('h3 a')['href']
            
            # Добавляем данные о книге в список
            books_data.append({
                'id': book_id,
                'book_name': book_name,
                'price': price,
                'stock': stock,
                'url': book_url
            })
    
    return books_data

# Получаем данные о книгах
books_data = scrape_books()

# Выводим первые 5 книг с полным URL для проверки
for book in books_data[:5]:
    print(book['book_name'], book['url'])

# Создаем DataFrame из данных о книгах
df = pd.DataFrame(books_data)

# Сохраняем DataFrame в CSV файл
df.to_csv('books_dataset.csv', index=False)

A Light in the Attic http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html
Tipping the Velvet http://books.toscrape.com/catalogue/tipping-the-velvet_999/index.html
Soumission http://books.toscrape.com/catalogue/soumission_998/index.html
Sharp Objects http://books.toscrape.com/catalogue/sharp-objects_997/index.html
Sapiens: A Brief History of Humankind http://books.toscrape.com/catalogue/sapiens-a-brief-history-of-humankind_996/index.html


## Итоговый датасет

In [3]:
df.shape

(1000, 5)

In [4]:
display(
    df.head(),
    df.tail()
)

,id,book_name,price,stock,url
0,1000/index,A Light in the Attic,51.77,1,http://books.toscrape.com/catalogue/a-light-in...
1,999/index,Tipping the Velvet,53.74,1,http://books.toscrape.com/catalogue/tipping-th...
2,998/index,Soumission,50.10,1,http://books.toscrape.com/catalogue/soumission...
3,997/index,Sharp Objects,47.82,1,http://books.toscrape.com/catalogue/sharp-obje...
4,996/index,Sapiens: A Brief History of Humankind,54.23,1,http://books.toscrape.com/catalogue/sapiens-a-...


,id,book_name,price,stock,url
995,5/index,Alice in Wonderland (Alice's Adventures in Won...,55.53,1,http://books.toscrape.com/catalogue/alice-in-w...
996,4/index,"Ajin: Demi-Human, Volume 1 (Ajin: Demi-Human #1)",57.06,1,http://books.toscrape.com/catalogue/ajin-demi-...
997,3/index,A Spy's Devotion (The Regency Spies of London #1),16.97,1,http://books.toscrape.com/catalogue/a-spys-dev...
998,2/index,1st to Die (Women's Murder Club #1),53.98,1,http://books.toscrape.com/catalogue/1st-to-die...
999,1/index,"1,000 Places to See Before You Die",26.08,1,http://books.toscrape.com/catalogue/1000-place...


# ЗАДАНИЕ ПРО

Так, мы спарсили данные о книгах. Но данные какие-то неполные. Часть названия стирается из-за отображения и нет ни полного названия книги, ни описания этой книги, ни жанра.

Вам необходимо дополнить датасет, спарсив дополнительные данные из того же сайта:

https://books.toscrape.com/
 
Итоговая таблица должна содержать следующие столбцы:

| Название столбца | Описание | 
|--|--|
|id| Идентификатор книги |
|book_name| Название книги - только полное название|
|genre| жанр книги |
|desc| описание |
|price| Цена в £ |
|stock| Наличие книги. 1 или 0|
|url| Ссылка на книгу |
| num_of_rev | количество отзывов|

## Парсинг данных и обогащение датасета

In [14]:
# Функция для получения данных о книгах
def scrape_books():
    # Создаем пустой список для хранения данных о книгах
    books_data = []
    
    # Перебираем все страницы с книгами (50 страниц)
    for page_num in range(1, 51):
        url = f'http://books.toscrape.com/catalogue/page-{page_num}.html'
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Находим все элементы с информацией о книгах на странице
        books = soup.find_all('article', class_='product_pod')
        
        # Извлекаем данные о каждой книге
        for book in books:
            # Извлекаем id книги
            book_id = book.select_one('h3 a')['href'].split('_')[-1].split('.')[0]
            
            # Извлекаем название книги
            book_name = book.select_one('h3 a')['title']
            
            # Извлекаем цену книги и удаляем лишние символы
            price_str = book.select_one('p.price_color').get_text().replace('£', '').replace('Â', '')
            price = float(price_str)
            
            # Извлекаем наличие книги
            if 'In stock' in book.select_one('p.availability').get_text():
                stock = 1
            else:
                stock = 0
            
            # Создаем ссылку на книгу
            book_url = 'http://books.toscrape.com/catalogue/' + book.select_one('h3 a')['href']
            
            # Получаем страницу с подробной информацией о книге
            book_response = requests.get(book_url)
            book_soup = BeautifulSoup(book_response.text, 'html.parser')
            
            # Извлекаем жанр книги
            genre = book_soup.select_one('ul.breadcrumb').select('li')[2].get_text().strip()
            
            # Извлекаем описание книги
            desc = book_soup.find('meta', attrs={'name': 'description'})['content']
            
            # Извлекаем количество отзывов
            num_of_rev_element = book_soup.select_one('p.star-rating')
            num_of_rev = 0
            if num_of_rev_element:
                num_of_rev_class = num_of_rev_element.get('class')
                if num_of_rev_class and len(num_of_rev_class) > 1:
                    num_of_rev_value = num_of_rev_class[1].split('-')
                    if len(num_of_rev_value) > 1:
                        num_of_rev = int(num_of_rev_value[1])
            
            # Добавляем данные о книге в список
            books_data.append({
                'id': book_id,
                'book_name': book_name,
                'genre': genre,
                'desc': desc,
                'price': price,
                'stock': stock,
                'url': book_url,
                'num_of_rev': num_of_rev
            })
    
    return books_data

# Получаем данные о книгах
books_data = scrape_books()

# Создаем DataFrame из данных о книгах
df = pd.DataFrame(books_data)

# Сохраняем DataFrame в CSV файл
df.to_csv('books_dataset_with_extra_info.csv', index=False)

## Итоговый датасет PRO

In [15]:
df.shape

(1000, 8)

In [16]:
display(
    df.head(),
    df.tail()
)

,id,book_name,genre,desc,price,stock,url,num_of_rev
0,1000/index,A Light in the Attic,Poetry,\n It's hard to imagine a world without A L...,51.77,1,http://books.toscrape.com/catalogue/a-light-in...,0
1,999/index,Tipping the Velvet,Historical Fiction,"\n ""Erotic and absorbing...Written with sta...",53.74,1,http://books.toscrape.com/catalogue/tipping-th...,0
2,998/index,Soumission,Fiction,\n Dans une France assez proche de la nÃ´tr...,50.10,1,http://books.toscrape.com/catalogue/soumission...,0
3,997/index,Sharp Objects,Mystery,"\n WICKED above her hipbone, GIRL across he...",47.82,1,http://books.toscrape.com/catalogue/sharp-obje...,0
4,996/index,Sapiens: A Brief History of Humankind,History,\n From a renowned historian comes a ground...,54.23,1,http://books.toscrape.com/catalogue/sapiens-a-...,0


,id,book_name,genre,desc,price,stock,url,num_of_rev
995,5/index,Alice in Wonderland (Alice's Adventures in Won...,Classics,\n \n,55.53,1,http://books.toscrape.com/catalogue/alice-in-w...,0
996,4/index,"Ajin: Demi-Human, Volume 1 (Ajin: Demi-Human #1)",Sequential Art,\n High school student Kei Nagai is struck ...,57.06,1,http://books.toscrape.com/catalogue/ajin-demi-...,0
997,3/index,A Spy's Devotion (The Regency Spies of London #1),Historical Fiction,"\n In Englandâs Regency era, manners and ...",16.97,1,http://books.toscrape.com/catalogue/a-spys-dev...,0
998,2/index,1st to Die (Women's Murder Club #1),Mystery,"\n James Patterson, bestselling author of t...",53.98,1,http://books.toscrape.com/catalogue/1st-to-die...,0
999,1/index,"1,000 Places to See Before You Die",Travel,"\n Around the World, continent by continent...",26.08,1,http://books.toscrape.com/catalogue/1000-place...,0
